In [1]:
import pandas as pd
import json
import requests
import bs4

In [2]:
""" Analyse de la page HTML atlashymenoptera.net pour récupérer les liens de toutes les sources de données pour les différentes espèces """

def get_all_data_sources():
    # html page bombus dataset
    page_url = 'http://www.atlashymenoptera.net/page.aspx?id=169'
    page_text = requests.get(page_url).text
    soup = bs4.BeautifulSoup(page_text, 'html.parser')
    
    data_sources = []

    # Analyse des éléments de DOM ayant la classe mt-3
    for row in soup.find_all(class_='mt-3'):
        # Le nom de l'espèce est contenu dans l'élément de classe nomtaxon
        specie_name = row.find(class_='nomtaxon')
        if specie_name is None: continue
        # Le lien vers la source de données est contenu dans un sous-élements <a> du sous-éléement de classe col-lg-2
        link = row.find(class_='col-lg-2').find('a', text='Download dataset')
        if link is None: continue
        # Enlever les cactères blancs sur le nom de l'espèce
        specie_name = specie_name.text.strip()
        # Récupération des liens vers les images : ils sont contenus dans des sous-éléments de classe mb-3
        next_row = row.findNext(class_='mb-3').find_all('img')
        imgs = [nr.get('src') for nr in next_row]
        # Lien de sources rattachée à une espèce (identifiant espèce + nom de l'espèce + lien images + liens CSV )
        data_sources.append({
            'link': link.get('href'),
            'imgs': imgs,
            'name': specie_name,
            'id': len(data_sources) + 1
        })

    return data_sources

In [3]:
"""  # Affichage de la répartition par année (pour debug) """
def displayContentByYear():
    testByYear = {}
    print(testByYear.keys())
    for row in cleaned_dataset:
        year1 = row[0] + "#" + row[3]
        year1 = row[0]
        #print("year1 : ", year1)
        if(year1 in testByYear.keys()):
            testByYear[year1]+= 1
        else:
            testByYear[year1] = 1
        #print(row)
    testByYear2 = sorted(testByYear.items(), key=lambda kv: -1*kv[1])
    print(testByYear2)


def fetch_all_dataset(data_sources):
    # données de l'ensemble des espèces
    cleaned_dataset = []
    # Parccourrir les liens que l'on a sur les différentes espèces
    for source in data_sources:
        if(1*source['id']>=9999):
            continue
        server = 'http://www.atlashymenoptera.net/'
        # Récupérer le lien du fichier csv de la prochaine espèce
        dataset_url = server + source['link']
        #print("dataset_url for specie ", source['id']," : ",  dataset_url)
        # Charger le fichier csv de la prochaine espèce
        dataset = pd.read_csv(dataset_url, delimiter='\t', skiprows=2)
        dataset = dataset.drop(['Authors', 'Pub_year', 'Source'], axis=1)
        # Récupérer les données de ce CSV et les ajouter aux données de l'ensemble des espèces 
        cleaned_dataset += clean_dataset(dataset, source['id'])
        print('------ finished', source['name'], 'id', source['id'], '---------')
    # displayContentByYear()
    return pd.DataFrame(cleaned_dataset, columns=['Year', 'Latitude', 'Longitude', 'SpecieId', 'Frequency'])

In [4]:

""" Vérifie si le format de l'année est correct ou non """
def invalide_year(data):    
    year=data[0]
    if(isinstance(year, float)):
        # Nombre déjà au foramt décimale (éviter un appel à strip() qui renvoie une exception)
        # Vérifier que la partie décimale est nulle
        yearInt = int(year)
        #print("invalide_year isflotat ", data, (yearInt  == year))
        return (yearInt  == year) and yearInt > 0
    # Supprimer les espaces
    year2 = year.strip() 
    if year2 == '': return True
    if not str.isdigit(year2): return True
    yearInt = int(year2)
    # L'année doit être comprise entre 1800 et 2100
    if(yearInt>2100 or yearInt<1800): return True
    return False


""" Récupération des données d'une espèce à partir du contenu du fichier csv """
def clean_dataset(dataset, specie_id):
    
    # Table des fréquences par année et position géographique et espèce
    frequencies_dict = {}

    for data in dataset.values:

        if invalide_year(data): continue
            
        # Clé d'une donnée élémentaire : année + position géogrpahique + identifiant de l'espèce
        key = '{} {} {} {}'.format(int(data[0]), data[1], data[2], specie_id)

        # Mise à jour de la fréquence associé à la clé
        if key in frequencies_dict.keys():
            # Clé déjà rencontrée dans le csv : on incrémente la fréquence associée
            frequencies_dict[key] += 1

        else: frequencies_dict[key] = 1

    frequencies = []

    for freq in frequencies_dict.keys():
        k = freq.split(' ')
        k.append(frequencies_dict[freq])
        frequencies.append(k)
    return frequencies
    # frequencies = pd.DataFrame(frequencies, columns=['Year', 'Latitude', 'Longitude', 'SpecieId', 'Frequency'])
    
    

In [5]:
data_sources = get_all_data_sources()
dataset = fetch_all_dataset(data_sources)


dataset.to_csv('bombus_terrestris_freqs.csv.gz', index=False, compression='gzip')
with open('data_sources.json', 'w+') as output_file:
    json.dump(data_sources, output_file)
print('finished download')

------ finished Bombus (Alpigenobombus) wurflenii id 1 ---------
------ finished Bombus (Alpinobombus) alpinus id 2 ---------
------ finished Bombus (Alpinobombus) balteatus id 3 ---------
------ finished Bombus (Alpinobombus) hyperboreus id 4 ---------
------ finished Bombus (Alpinobombus) polaris id 5 ---------
------ finished Bombus (Bombus) cryptarum id 6 ---------
------ finished Bombus (Bombus) lucorum id 7 ---------
------ finished Bombus (Bombus) magnus id 8 ---------
------ finished Bombus (Bombus) patagiatus id 9 ---------
------ finished Bombus (Bombus) sporadicus id 10 ---------
------ finished Bombus (Bombus) terrestris id 11 ---------
------ finished Bombus (Confusibombus) confusus id 12 ---------
------ finished Bombus (Cullumanobombus) cullumanus id 13 ---------
------ finished Bombus (Cullumanobombus) semenoviellus id 14 ---------
------ finished Bombus (Kallobombus) soroeensis id 15 ---------
------ finished Bombus (Megabombus) argillaceus id 16 ---------
------ finis